## Lesson 3: Reflection and Blogpost Writing

https://learn.deeplearning.ai/courses/ai-agentic-design-patterns-with-autogen/lesson/4/reflection-and-blogpost-writing

In [4]:
import autogen
from dotenv import find_dotenv, load_dotenv

In [5]:
load_dotenv(find_dotenv())

MODEL_NAME = "glm-4"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

llm_config = {"model": MODEL_NAME, "base_url": BASE_URL, "temperature": 0.5}

### The task!

In [6]:
task = """
    Write a concise but engaging blogpost about DeepLearning.AI.
    Make sure the blogpost is within 100 words.
"""

### Create a writer agent

In [7]:
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="""
        You are a writer. You write engaging and concise
        blogpost (with title) on given topics. You must polish your
        writing based on the feedback you receive and give a refined
        version. Only return your final work without additional comments.
    """,
)

In [8]:
reply = writer.generate_reply(messages=[{"role": "user", "content": task}])

In [9]:
print(reply)

Title: Unleashing the Power of DeepLearning.AI

DeepLearning.AI: a revolution in the world of artificial intelligence. This cutting-edge platform, founded by AI pioneer Andrew Ng, offers comprehensive courses and resources to master neural networks and deep learning. Whether you're a beginner or an experienced professional, DeepLearning.AI empowers you to unlock the potential of AI, driving innovation and transforming industries. Don't miss out on this opportunity to become a part of the deep learning revolution. Join DeepLearning.AI today and shape the future!


### Adding reflection
Create a critic agent to reflect on the work of the writer agent.

In [10]:
critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message="""
        You are a critic. You evaluate the content of the writer
        and give constructive feedback on the content to help improve
        the quality of the blogpost.
    """,
    is_termination_msg=lambda msg: msg.get("content", "").find("TERMINATE") >= 0,
)

In [11]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg",
)

Critic (to Writer):


    Write a concise but engaging blogpost about DeepLearning.AI.
    Make sure the blogpost is within 100 words.


--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of DeepLearning.AI

DeepLearning.AI: a revolution in the world of artificial intelligence. This cutting-edge platform, founded by AI pioneer Andrew Ng, offers comprehensive courses and resources to master neural networks and deep learning. Whether you're a beginner or an experienced professional, DeepLearning.AI empowers you to unlock the potential of AI, driving innovation and transforming industries. Don't miss out on this opportunity to become a part of the deep learning revolution. Join DeepLearning.AI today and shape the future!

--------------------------------------------------------------------------------
Critic (to Writer):

Your blogpost is concise and to the point, which is great for capturing attention in a bus

In [12]:
print(res.summary)

Title: Elevate Your AI Knowledge with DeepLearning.AI

Are you eager to explore the boundless world of artificial intelligence? DeepLearning.AI, under the guidance of AI luminary Dr. Andrew Ng, is your key to unlocking the secrets of neural networks and deep learning. This treasure trove of interactive courses promises to equip you with the expertise needed to drive AI innovation. Whether you seek to grasp the basics or delve into advanced concepts, DeepLearning.AI is your springboard to the cutting edge. Take the leap today. Join us and become a force in the AI revolution!


### Nested chat

In [13]:
seo_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="""
        You are a SEO reviewer, known for
        your ability to optimize content for search engines,
        ensuring that it ranks well and attracts organic traffic.
        Make sure your suggestion is concise (within 3 bullet points),
        concrete and to the point.
        Begin the review by stating your role.
    """,
)

legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="""
        You are a Legal reviewer, known for
        your ability to ensure that content is legally compliant
        and free from any potential legal issues.
        Make sure your suggestion is concise (within 3 bullet points),
        concrete and to the point.
        Begin the review by stating your role.
    """,
)

ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="""
        You are an Ethics reviewer, known for
        your ability to ensure that content is ethically sound
        and free from any potential ethical issues.
        Make sure your suggestion is concise (within 3 bullet points),
        concrete and to the point.
        Begin the review by stating your role.
    """,
)

meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="""
        You are a meta reviewer, you aggragate and review
        the work of other reviewers and give a final suggestion on the content.
    """,
)

### Orchestrate the nested chats to solve the task

In [14]:
def reflection_message(recipient, messages, sender, config):
    return f"""Review the following content.
        {recipient.chat_messages_for_summary(sender)[-1]["content"]}
    """

In [15]:
review_chats = [
    {
        "recipient": seo_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": """Return review into as JSON object only:
                {"Reviwer": "", "Review": ""}. Here Reviewer should be your role.
            """,
        },
        "max_turns": 1,
    },
    {
        "recipient": legal_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": """Return review into as JSON object only:
                {"Reviwer": "", "Review": ""}.
            """,
        },
        "max_turns": 1,
    },
    {
        "recipient": ethics_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": """Return review into as JSON object only:
                {"reviwer": "", "review": ""}.
            """,
        },
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregate feedback from all reviewers and give final suggestions on the writting.",
        "max_turns": 1,
    },
]

In [16]:
critic.register_nested_chats(chat_queue=review_chats, trigger=writer)

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg",
)

Critic (to Writer):


    Write a concise but engaging blogpost about DeepLearning.AI.
    Make sure the blogpost is within 100 words.


--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of DeepLearning.AI

DeepLearning.AI: a revolution in the world of artificial intelligence. This cutting-edge platform, founded by AI pioneer Andrew Ng, offers comprehensive courses and resources to master neural networks and deep learning. Whether you're a beginner or an experienced professional, DeepLearning.AI empowers you to unlock the potential of AI, driving innovation and transforming industries. Don't miss out on this opportunity to become a part of the deep learning revolution. Join DeepLearning.AI today and shape the future!

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

****

### Get the summary

In [18]:
print(res.summary)

Title: Unleashing the Power of DeepLearning.AI

Embark on a comprehensive learning path with DeepLearning.AI to explore the vast potential of AI and contribute to technological innovation. Founded by the renowned AI expert, Andrew Ng, this educational platform offers a goldmine of resources for learners of all levels. Start your AI journey now and be part of a transformative wave in neural networks and deep learning. Remember, with DeepLearning.AI, you're not just learning; you're shaping the future. [Disclosure: This post may contain affiliate links.]
